In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np

/home/leoxing/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# output = slim.conv2d(input, 
#                      output_channels, 
#                      [kernel_height, kernel_width],
#                      stride=1,
#                      padding='SAME')

# w = slim.model_variable(name, 
#                         shape=[])

In [3]:
def res_block_same(input_image, kernel_num, kernel_size, name):
    '''
    尺寸不变的残差链接
    '''
    with tf.name_scope(name):
        # first layer
        first_output = tf.nn.relu(slim.conv2d(input_image, kernel_num, kernel_size))
        # second layer
        second_output = slim.conv2d(first_output, kernel_num, kernel_size)
        
        output_image = tf.nn.relu(second_output + input_image)
    return output_image

def res_block_increase(input_image, kernel_num, kernel_size, name):
    '''
    尺寸改变的残差链接
    '''
    with tf.name_scope(name):
        # first layer
        first_output = tf.nn.relu(slim.conv2d(input_image, kernel_num, kernel_size, stride=2))
        
        # second layer
        second_output = slim.conv2d(first_output, kernel_num, kernel_size)
        
        # reshape input
        input_reshape = slim.conv2d(input_image, kernel_num, [1,1], stride=2, padding='VALID')
        
        output_image = tf.nn.relu(second_output + input_reshape)
    return output_image

In [4]:
mnist = input_data.read_data_sets('MINST_data')

# mnist shape:
# pic : batch * 784
# label: batch * 0

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MINST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
image_shape = mnist.train.images.shape[1]
width = np.sqrt(image_shape)
height = np.sqrt(image_shape)
channel = 1

BATCH_SIZE = 50
KERNEL_SIZE = [3, 3]

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.0001

input_image = tf.placeholder(tf.float32, 
                             shape=[None, height, width, channel], 
                             name="Input_Image")

labels = tf.placeholder(tf.float32,
                        shape=[None, 10],
                        name="Labels")

output_temp_1 = res_block_same(input_image, 3, 
                               KERNEL_SIZE, "output_temp_1") # 28 * 28 * 1

output_temp_2 = res_block_increase(output_temp_1, 6, 
                                   KERNEL_SIZE, "output_temp_2") # 14 * 14 * 6

output_temp_3 = res_block_same(output_temp_2, 6,
                              KERNEL_SIZE, "output_temp_3") # 14 * 14 * 6

output_temp_4 = res_block_increase(output_temp_3, 12,
                                  KERNEL_SIZE, "output_temp_4") # 7 * 7 * 12

# AVG_POOL ---> 'SAME'  这里 ksize 似乎必须显式声明 batch size
# 去掉池化之后效果变好了...
# 似乎因为损失的数据变少了
output_pool = tf.nn.avg_pool(output_temp_4, ksize=[BATCH_SIZE, 3, 3, 12], strides=[1,1,1,1], padding='SAME')
output_pool = tf.reshape(output_temp_4, [-1, 7*7*12])


# 1层FC
w = slim.variable("Full_Connect_Weight", shape=[7*7*12, 10])
b = slim.variable("Full_Connect_Bias", shape=[BATCH_SIZE, 10])
output = tf.matmul(output_pool, w) + b

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=output))
labels_pre = tf.argmax(output)

global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step,
                                           mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY)

trainer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

accu_rate = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)), tf.float32))

saver = tf.train.Saver()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [7]:
training_steps = 10000
intervals = 100
PATH = 'MODEL/mnistRes.ckpt'

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    loss_min = 100
    for i in range(1, training_steps):
        batch = mnist.train.next_batch(50)
        input_image_ = batch[0].reshape(BATCH_SIZE, 28, 28, 1)
        labels_ = tf.one_hot(batch[1], depth=10).eval()
        
        usedData = {
            input_image: input_image_,
            labels: labels_ 
        }
        
        trainer.run(session=sess, feed_dict=usedData)
        if i % intervals == 0:
            accu_rate_ = accu_rate.eval(session=sess, feed_dict=usedData)
            loss_ = loss.eval(session=sess, feed_dict=usedData)
            print("Step %4d      Accuracy: %5g   Loss: %5g" %(i, accu_rate_, loss_))
            
            if loss_ < loss:
                saver.save(sess, PATH, global_step=i)
            

Step   50      Accuracy:  0.76   Loss: 0.49933
Step  100      Accuracy:  0.94   Loss: 0.192217
Step  150      Accuracy:  0.96   Loss: 0.230655
Step  200      Accuracy:  0.92   Loss: 0.363342
Step  250      Accuracy:  0.88   Loss: 0.467538
Step  300      Accuracy:  0.82   Loss: 0.574525
Step  350      Accuracy:  0.82   Loss: 0.543559
Step  400      Accuracy:  0.82   Loss: 0.415361
Step  450      Accuracy:   0.8   Loss: 0.617212
Step  500      Accuracy:  0.92   Loss: 0.256725
Step  550      Accuracy:   0.9   Loss: 0.331106
Step  600      Accuracy:  0.86   Loss: 0.544242
Step  650      Accuracy:   0.9   Loss: 0.362776
Step  700      Accuracy:  0.92   Loss: 0.243449
Step  750      Accuracy:  0.76   Loss: 0.573966
Step  800      Accuracy:  0.88   Loss: 0.395885
Step  850      Accuracy:  0.94   Loss: 0.303172
Step  900      Accuracy:  0.86   Loss: 0.49256
Step  950      Accuracy:  0.84   Loss: 0.335808


In [52]:
# Pooling
# tf.nn.avg_pool(value, ksize, strides, padding)
# value 为输入 形如 [batch height width channels] 的 4-D tensor 直接写一个二维数组然后 reshape 就行
# ksize 为池化尺寸 形如 [batch height width channels] 的 1-D tensor
# strides 池化步长 形式与 ksize 相同
# padding 有 SAME 和 VALID 两个选项


t = np.array([[1,2,3],[2,3,1],[3,1,2]])
t.shape

t_ = tf.constant(t.reshape([1,3,3,1]), dtype=tf.float32)
res = tf.nn.avg_pool(t_, ksize=[1,2,2,1], strides=[1,1,1,1], padding='VALID')
res_ = tf.nn.avg_pool(t_, ksize=[1,2,2,1], strides=[1,1,1,1], padding='SAME')
with tf.Session() as sess:
    print(res.eval())
    print(res_.eval())
    

# 多 Channel 的情况 

t1 = np.array([[1,2,3],[2,3,1],[3,1,2]])
t2 = np.array([[1,2,3],[2,3,1],[3,1,2]])

tt = np.array([     # tt 为 2 channels
    [t1[i][j], t2[i][j]] for i in range(3) for j in range(3)
]).reshape([1,3,3,2])

tt = tf.constant(tt, dtype=tf.float32)

Res = tf.nn.avg_pool(tt, ksize=[1,2,2,1], strides=[1,1,1,1], padding='VALID')

with tf.Session() as sess:
    print(Res.eval())
    print(tt.eval())

[[[[2.  ]
   [2.25]]

  [[2.25]
   [1.75]]]]
[[[[2.  ]
   [2.25]
   [2.  ]]

  [[2.25]
   [1.75]
   [1.5 ]]

  [[2.  ]
   [1.5 ]
   [2.  ]]]]
[[[[2.   2.  ]
   [2.25 2.25]]

  [[2.25 2.25]
   [1.75 1.75]]]]
[[[[1. 1.]
   [2. 2.]
   [3. 3.]]

  [[2. 2.]
   [3. 3.]
   [1. 1.]]

  [[3. 3.]
   [1. 1.]
   [2. 2.]]]]


In [9]:
tf.VERSION

'1.8.0'